In [24]:
import nb_utils

In [25]:
import torch
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
from mle import bootstrap_intrinsic_dim_scale_interval as mle

In [29]:
#Import R packages
import rpy2
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
pandas2ri.activate()

from rpy2.robjects.packages import importr
r_base = importr('base')
intdimr = importr('intrinsicDimension')

In [34]:
from lightning_data_modules import HaarDecomposedDataset, ImageDatasets, PairedDataset, SyntheticDataset, SyntheticPairedDataset, Synthetic1DConditionalDataset, SyntheticTimeSeries, SRDataset, SRFLOWDataset, CryptoDataset, KSphereDataset, MammothDataset, LineDataset

from configs.mammoth.vesde import get_config
#from configs.ksphere.N_1.uniform import get_config
#from configs.line.vesde import get_config

config = get_config()

In [35]:
from lightning_data_modules.utils import create_lightning_datamodule
DataModule = create_lightning_datamodule(config)
DataModule.setup()
train_dataloader = DataModule.train_dataloader()

In [36]:
X=[]
for _, x in enumerate(train_dataloader):
    X.append(x)
X = torch.cat(X, dim=0).numpy()
X = pd.DataFrame(X)

# MLE

In [26]:
k = 5
mle_result = mle(X,  k1=k, k2=k, average=True)[0]

In [28]:
mle_result

array([35.23894551])

# MLE R

# Local PCA

In [37]:
data_r = X
dim = X.shape[1]
lpca = intdimr.pcaLocalDimEst(data_r, 'FO').rx2('dim.est')[0]

In [38]:
lpca

3

In [39]:
mle = intdimr.maxLikLocalDimEst(data_r, K=5).rx2('dim.est')[0]

In [40]:
mle

1.0418394247586638